In [1]:
import pandas as pd 
import re

In [2]:
df = pd.read_csv('IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


<H1> Data Preprocessing </H1>

In [4]:
#Replace sentiment with 0 and 1
df['sentiment'].replace({'positive':1,'negative':0},inplace=True)

In [5]:
#Clean reviews
import re
clean = re.compile('<.*?>')
re.sub(clean,'',df.iloc[1].review)
def clean_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean,'',text)
df['review'] = df['review'].apply(clean_html)

#Convert chars to lowercase
def convert_lower(text):
    return text.lower()
df['review'] = df['review'].apply(convert_lower)

#Remove special characters (non-alphanumeric)
def remove_special(text):
    x=''
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x=x+' '
    return x
df['review'] = df['review'].apply(remove_special)
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there s a family where a little boy ...,0
4,petter mattei s love in the time of money is...,1


In [6]:
print("Shape of df: ", df.shape)

Shape of df:  (50000, 2)


In [7]:
df.dropna(inplace = True)

In [8]:
print("Shape of df: ", df.shape)

Shape of df:  (50000, 2)


<b> Train-val-test split </b>

In [9]:
from sklearn.model_selection import train_test_split 

# determine X_test & y_test 
X_temp, X_test, y_temp, y_test = train_test_split(df['review'], df['sentiment'], test_size = 0.2, random_state = 42)

X_temp = X_temp.reset_index(drop = True)
y_temp = y_temp.reset_index(drop = True)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size = 0.125, random_state = 42)

<H1> BERT Encoding (Embedding) </H1>

Output to CSV

In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [11]:
print('encode val ...')
val_encoded = model.encode(X_val.tolist())
dfe_val = pd.DataFrame(val_encoded)
y_val1 = y_val.reset_index(drop = True, inplace = False)
dfe_val['sentiment'] = y_val1
dfe_val.to_csv("dfe_val.csv", index = False)

encode val ...


In [12]:
print('encode test ...')
test_encoded = model.encode(X_test.tolist())
dfe_test = pd.DataFrame(test_encoded)
y_test1 = y_test.reset_index(drop = True, inplace = False)
dfe_test['sentiment'] = y_test1
dfe_test.to_csv("dfe_test.csv", index = False)

encode test ...


In [13]:
# encode
print('encode train ...')
train_encoded = model.encode(X_train.tolist())
dfe_train = pd.DataFrame(train_encoded)
y_train1 = y_train.reset_index(drop = True, inplace = False)
dfe_train['sentiment'] = y_train1
dfe_train.to_csv("dfe_train.csv", index = False)

encode train ...
